# 惡性腫瘤分類問題（二元分類）

使用 Logistic Regression Classifier 及 Stochastic Gradient Descent Classifier 來解二元分類問題。

我們使用的是 Donald Bren School of Information and Computer Sciences 分享的資料集，這個資料集是用來訓練良/惡性乳癌腫瘤預測模型的，下載網址在：https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/

其中資料集是：https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/breast-cancer-wisconsin.data

至於資料集中每筆資料及欄位的資訊大家可以查看這個說明：https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/breast-cancer-wisconsin.names

接下來我們就來看看怎麼訓練一個二元分類器吧！

## Step1 載入資料集

In [1]:
# 引入 pandas 及 numpy
import pandas as pd
import numpy as np

In [2]:
# 將特徵列表記下來，讓我們資料可以方便使用
column_names = ['Sample code number', 'Clump Thickness', 
                'Uniformity of Cell Size', 'Uniformity of Cell Shape', 
                'Marginal Adhesion', 'Single Epithelial Cell Size',
                'Bare Nuclei', 'Bland Chromatin',
                'Normal Nucleoli', 'Mitoses', 'Class']

In [3]:
column_names

['Sample code number',
 'Clump Thickness',
 'Uniformity of Cell Size',
 'Uniformity of Cell Shape',
 'Marginal Adhesion',
 'Single Epithelial Cell Size',
 'Bare Nuclei',
 'Bland Chromatin',
 'Normal Nucleoli',
 'Mitoses',
 'Class']

In [4]:
# 使用 pandas 直接讀取網路上的資料集
data = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/breast-cancer-wisconsin.data', names=column_names)

In [5]:
data

,Sample code number,Clump Thickness,Uniformity of Cell Size,Uniformity of Cell Shape,Marginal Adhesion,Single Epithelial Cell Size,Bare Nuclei,Bland Chromatin,Normal Nucleoli,Mitoses,Class
0,1000025,5,1,1,1,2,1,3,1,1,2
1,1002945,5,4,4,5,7,10,3,2,1,2
2,1015425,3,1,1,1,2,2,3,1,1,2
3,1016277,6,8,8,1,3,4,3,7,1,2
4,1017023,4,1,1,3,2,1,3,1,1,2
5,1017122,8,10,10,8,7,10,9,7,1,4
6,1018099,1,1,1,1,2,10,3,1,1,2
7,1018561,2,1,2,1,2,1,3,1,1,2
8,1033078,2,1,1,1,2,1,1,1,5,2
9,1033078,4,2,1,1,2,1,2,1,1,2


In [6]:
# 處理缺失值，將被標示為 ? 的資料換成 nan
data = data.replace(to_replace='?', value=np.nan)
# 丟棄帶有缺失值的資料（只要有一個維度有缺失值就丟掉）
data = data.dropna(how='any')

In [7]:
data

,Sample code number,Clump Thickness,Uniformity of Cell Size,Uniformity of Cell Shape,Marginal Adhesion,Single Epithelial Cell Size,Bare Nuclei,Bland Chromatin,Normal Nucleoli,Mitoses,Class
0,1000025,5,1,1,1,2,1,3,1,1,2
1,1002945,5,4,4,5,7,10,3,2,1,2
2,1015425,3,1,1,1,2,2,3,1,1,2
3,1016277,6,8,8,1,3,4,3,7,1,2
4,1017023,4,1,1,3,2,1,3,1,1,2
5,1017122,8,10,10,8,7,10,9,7,1,4
6,1018099,1,1,1,1,2,10,3,1,1,2
7,1018561,2,1,2,1,2,1,3,1,1,2
8,1033078,2,1,1,1,2,1,1,1,5,2
9,1033078,4,2,1,1,2,1,2,1,1,2


In [8]:
# 使用 sklearn.cross_validation 的 train_test_split 分割資料集，一部分做為訓練，一部分作為測試
from sklearn.cross_validation import train_test_split

/Users/fukuball/Projects/Hands-on-AI-Tools-env/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [9]:
X_train, X_test, y_train, y_test = train_test_split(
    data[column_names[1:10]], data[column_names[10]], test_size=0.25, random_state=33
)

In [10]:
# 查看訓練樣本的數量及類別分佈
y_train.value_counts()

2    344
4    168
Name: Class, dtype: int64

In [11]:
# 查看測試樣本的數量及類別分佈
y_test.value_counts()

2    100
4     71
Name: Class, dtype: int64

## Step2 資料前置處理（標準化）

In [12]:
# 從 sklearn.preprocessing 導入 StandardScaler
from sklearn.preprocessing import StandardScaler

In [13]:
# 從 sklearn.linear_model 導入 LogisticRegression 與 SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier

In [14]:
# 標準化數據，保證每個維度的特徵數據方差為 1，均值為 0，讓預測結果不會被某些維度過大的特徵值而主導
ss = StandardScaler()
X_train = ss.fit_transform(X_train)
X_test = ss.transform(X_test)

In [15]:
X_train

array([[-1.19196677, -0.68788787, -0.03809494, ..., -0.55666054,
        -0.58626819, -0.34343195],
       [-1.19196677,  0.30608745, -0.03809494, ...,  0.68611648,
         1.09441105, -0.34343195],
       [ 0.24231522, -0.02523765, -0.72246721, ..., -0.97091955,
        -0.58626819, -0.34343195],
       ..., 
       [ 0.24231522, -0.68788787, -0.38028108, ..., -0.97091955,
        -0.58626819, -0.34343195],
       [-1.19196677, -0.68788787, -0.72246721, ..., -0.55666054,
        -0.58626819, -0.34343195],
       [ 0.95945621, -0.02523765, -0.38028108, ...,  0.68611648,
         0.42213936,  1.42674078]])

## Step3 訓練（使用 Logistic Regression Classifier）

In [16]:
# 初始化 LogisticRegression
lr = LogisticRegression()

In [17]:
# 調用 fit 函式訓練預測模型
lr.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

## Step4 預測

In [18]:
# 使用訓練好的模型 lr 對 X_test 進行預測
lr_y_predict = lr.predict(X_test)

In [19]:
lr_y_predict

array([2, 2, 4, 4, 2, 2, 2, 4, 2, 2, 2, 2, 4, 2, 4, 4, 4, 4, 4, 2, 2, 4, 4,
       2, 4, 4, 2, 2, 4, 4, 4, 4, 4, 4, 4, 4, 2, 4, 4, 4, 4, 4, 2, 4, 2, 2,
       4, 2, 2, 4, 4, 2, 2, 2, 4, 2, 2, 2, 2, 2, 4, 4, 2, 2, 2, 4, 2, 2, 2,
       2, 4, 2, 2, 2, 2, 2, 2, 4, 4, 2, 2, 2, 4, 2, 2, 2, 4, 2, 4, 2, 4, 4,
       2, 2, 2, 2, 4, 4, 2, 2, 2, 4, 2, 2, 4, 2, 2, 2, 2, 2, 4, 2, 2, 2, 2,
       2, 2, 4, 2, 2, 4, 4, 2, 4, 2, 2, 2, 4, 2, 2, 4, 4, 2, 4, 4, 2, 2, 2,
       2, 4, 2, 4, 2, 4, 2, 2, 2, 2, 2, 4, 4, 2, 4, 4, 2, 4, 2, 2, 2, 2, 4,
       4, 4, 2, 4, 2, 2, 4, 2, 4, 4])

In [20]:
# 肉眼對答案
y_test

657    2
257    2
236    4
697    4
599    2
583    2
373    2
316    4
589    2
4      2
629    2
331    2
288    4
364    2
479    4
319    2
32     4
155    4
112    4
415    2
8      2
283    4
238    4
274    2
323    4
586    4
120    2
380    2
262    4
361    4
      ..
658    4
180    2
633    4
241    2
575    2
652    2
471    2
476    2
187    4
522    4
547    2
611    4
206    4
89     2
106    4
654    2
298    2
690    2
314    2
68     4
300    4
210    4
628    2
109    4
622    2
108    2
221    4
499    2
465    4
330    4
Name: Class, Length: 171, dtype: int64

In [21]:
X_test[0]

array([ 0.24231522,  0.30608745,  0.64627733, -0.61921941,  2.17533331,
       -0.6689353 , -0.14240153,  1.09441105, -0.34343195])

In [22]:
predict = lr.predict([[ 0.24231522,  0.30608745,  0.64627733, -0.61921941,  2.17533331,
       -0.6689353 , -0.14240153,  1.09441105, -0.34343195]])

In [23]:
predict

array([2])

## Step 5 評估

In [24]:
# 導入 classification_report
from sklearn.metrics import classification_report

In [25]:
# 使用 score 函式計算預測結果評分
print('Accuracy of LR Classifier:', lr.score(X_test, y_test))

('Accuracy of LR Classifier:', 0.98830409356725146)


In [26]:
# 使用 classification_report 取得 precision recall f1-score 預測結果評分
print(classification_report(y_test, lr_y_predict, target_names=['Benign', 'Malignent']))

             precision    recall  f1-score   support

     Benign       0.99      0.99      0.99       100
  Malignent       0.99      0.99      0.99        71

avg / total       0.99      0.99      0.99       171



## 使用 Stochastic Gradient Descent Classifier 的範例

In [27]:
# 初始化 SGDClassifier
sgdc = SGDClassifier()

In [28]:
# 調用 fit 函式訓練預測模型
sgdc.fit(X_train, y_train)

/Users/fukuball/Projects/Hands-on-AI-Tools-env/lib/python2.7/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', max_iter=None, n_iter=None,
       n_jobs=1, penalty='l2', power_t=0.5, random_state=None,
       shuffle=True, tol=None, verbose=0, warm_start=False)

In [29]:
# 使用訓練好的模型 sgdc 對 X_test 進行預測
sgdc_y_predict = sgdc.predict(X_test)

In [30]:
# 使用 score 函式計算預測結果評分
print('Accuracy of SGD Classifier:', sgdc.score(X_test, y_test))

('Accuracy of SGD Classifier:', 0.91228070175438591)


In [31]:
# 使用 classification_report 取得 precision recall f1-score 預測結果評分
print(classification_report(y_test, sgdc_y_predict, target_names=['Benign', 'Malignent']))

             precision    recall  f1-score   support

     Benign       0.88      0.98      0.93       100
  Malignent       0.97      0.82      0.89        71

avg / total       0.92      0.91      0.91       171



## 儲存訓練好模型

In [32]:
import pickle
pickle.dump(lr, open("model/breast-cancer-lr.p", "wb"))

## 載入訓練好模型，未來用於預測

In [33]:
lr_load = pickle.load(open("model/breast-cancer-lr.p", "rb"))
lr_load.predict([[ 0.24231522,  0.30608745,  0.64627733, -0.61921941,  2.17533331,
       -0.6689353 , -0.14240153,  1.09441105, -0.34343195]])

array([2])